# L2 Prompt Guidelines

## Setup


In [1]:
# 2024.07.31: 구글 코립일 경우 패키지 설치해야 함
!pip install -qU openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 389.3/389.3 kB 7.0 MB/s eta 0:00:00


In [5]:
import os
import openai

from openai import OpenAI
from google.colab import userdata

client = OpenAI(
    api_key=userdata.get('OPENAI_API_KEY')
)

#### helper function


In [3]:
def get_completion(prompt, model="gpt-3.5-turbo"):
    # 단일 메세지를 전달받아 message형식으로 구성
    messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0,
    )

    return response.choices[0].message.content

## 프롬프팅 원칙Prompting Principles
- **Principle 1: 분명하고 구체적인 지시 사항Write clear and specific instructions**
- **Principle 2: 모델에게 "생각"할 여유를 주기 Give the model time to “think”**




### Principle 1: 분명하고 구체적인 지시 사항

- 전략1: 지시하고자 하는 본문에 구분자 사용
    - `"""`, `'''`, ```<```, ```, `###`

- 전략2: 출력 형식을  HTML, JSON등으로 지정

- 전략3: 주어진 조건이 충족되었는지 확인하도록 요청

- 전략4: 가능하면 몇개의 예제를 프롬프트에 포함


#### 전략 1: 지시하고자 하는 본문에 구분자 사용
-  ```, """, < >, `<tag> </tag>`, `:`

In [7]:
text = f"""
모델이 수행하기를 원하는 작업을 다음과 같이 표현해야 합니다.
가능한 한 명확하고 구체적인 지침을 제공해야 합니다.
이렇게 하면 모델을 원하는 출력으로 유도할 수 있으며, 관련 없는 응답을 받을 가능성을 줄입니다.
또는 잘못된 응답을 받을 가능성을 줄입니다.
대부분의 경우, 긴 프롬프트가 더 명확하고 모델에게 풍부한 컨텍스트를 제공하기 때문에
더 상세하고 관련성 높은 출력을 얻을 수 있습니다.
"""

prompt = f"""
다음 `문자 세개로 감싸진 부분을 한 문장으로 요약하세요.
주의: 가능한 짧게 요약하고 반드시 한 문장에 되게 할 것

```{text}```
"""

response = get_completion(prompt)
print('gpt-3.5\n', response)

response = get_completion(prompt, model='gpt-4')
print('gpt-4\n', response)

response = get_completion(prompt, model='gpt-4o')
print('gpt-4o\n', response)


gpt-3.5
 모델이 원하는 작업을 명확하고 구체적으로 지시하면 더 정확하고 관련성 높은 출력을 얻을 수 있습니다.
gpt-4
 모델에게 원하는 작업을 명확하고 구체적으로 지시하면, 관련 없거나 잘못된 응답을 줄이고, 풍부한 컨텍스트를 제공하는 긴 프롬프트를 사용하면 더 상세하고 관련성 높은 출력을 얻을 수 있다.
gpt-4o
 명확하고 구체적인 지침을 제공하는 긴 프롬프트는 모델이 원하는 출력과 관련성 높은 응답을 얻는 데 도움이 됩니다.


In [8]:
text = f"""
모델이 수행하기를 원하는 작업을 다음과 같이 표현해야 합니다.
가능한 한 명확하고 구체적인 지침을 제공해야 합니다.
이렇게 하면 모델을 원하는 출력으로 유도할 수 있으며, 관련 없는 응답을 받을 가능성을 줄입니다.
또는 잘못된 응답을 받을 가능성을 줄입니다.
대부분의 경우, 긴 프롬프트가 더 명확하고 모델에게 풍부한 컨텍스트를 제공하기 때문에
더 상세하고 관련성 높은 출력을 얻을 수 있습니다.
"""

prompt = f"""
다음 # 문자 세개로 감싸진 부분을 한 문장으로 요약하세요.
주의: 가능한 짧게 요약하고 반드시 한 문장에 되게 할 것

###
{text}
###
"""

response = get_completion(prompt)
print('gpt-3.5\n', response)

response = get_completion(prompt, model='gpt-4')
print('gpt-4\n', response)

response = get_completion(prompt, model='gpt-4o')
print('gpt-4o\n', response)


gpt-3.5
 모델이 원하는 작업을 명확하고 구체적으로 설명하여 관련성 높은 출력을 얻을 수 있습니다.
gpt-4
 모델에게 원하는 작업을 수행하게 하려면 명확하고 구체적인 지침을 제공하며, 이는 관련 없거나 잘못된 응답을 줄이고, 풍부한 컨텍스트를 제공하는 긴 프롬프트를 통해 더 상세하고 관련성 높은 출력을 얻을 수 있게 합니다.
gpt-4o
 명확하고 구체적인 지침을 제공하면 모델이 원하는 출력을 생성하는 데 도움이 됩니다.


#### 전략 2: 출력 형식을 HTML, JSON등으로 지정
- JSON, HTML

In [9]:
format = 'JSON'

text = f"""
주제: 이미지 생성 AI
"""

prompt = f"""
다음 `문자 세개로 감싸진 부분을 주제로 하여 블로그 글을 작성하고 싶습니다.
다음 출력 포맷에 맞게 예를 3개 만들어 주세요.

주의: 반드시 예제 3개 출력할 것

출력 포맷
 - 출력에 포함될 내용: 글 제목, 관심을 끌만한 부제목, 해당 글의 카테고리
 - 출력 형식은 {format} 을 사용할 것
 - 각 항목의 JSON key 또는 HTML tag: title, subtitle, category

```{text}```
"""


In [10]:
response = get_completion(prompt, model='gpt-3.5-turbo-1106')
print('gpt-3.5-trubo\n', response)

print()

response = get_completion(prompt, model='gpt-4')
print('gpt-4\n', response)

print()

response = get_completion(prompt, model='gpt-4o')
print('gpt-4o\n', response)

gpt-3.5-trubo
 1. 
{
  "title": "AI가 빅데이터로 이미지 생성",
  "subtitle": "빅데이터를 활용한 AI 이미지 생성 기술의 혁신",
  "category": "인공지능"
}

2. 
{
  "title": "미래의 예술: AI로 만들어진 이미지",
  "subtitle": "인공지능이 만들어낸 예술적인 이미지의 매력",
  "category": "기술과 예술"
}

3. 
{
  "title": "AI가 그린 세상: 이미지 생성 기술의 가능성",
  "subtitle": "인공지능이 그리는 미래의 세상을 상상해보다",
  "category": "기술과 예술"
}

gpt-4
 예제 1:
```json
{
  "title": "AI가 그림을 그릴 수 있을까? 이미지 생성 AI에 대해 알아보자",
  "subtitle": "AI가 창작의 영역까지 넘보는 순간, 이미지 생성 AI의 등장",
  "category": "인공지능"
}
```

예제 2:
```json
{
  "title": "AI의 창작력을 확인하라! 이미지 생성 AI의 놀라운 세계",
  "subtitle": "AI가 그림을 그린다면? 이미지 생성 AI의 실제 성능을 분석해보자",
  "category": "기술"
}
```

예제 3:
```json
{
  "title": "이미지 생성 AI, 창작의 새로운 패러다임을 열다",
  "subtitle": "AI가 만든 그림, 진짜와 구별이 갈까? 이미지 생성 AI의 현실적인 성능과 한계",
  "category": "과학"
}
```

gpt-4o
 ```json
[
    {
        "title": "이미지 생성 AI: 창의성의 새로운 지평",
        "subtitle": "인공지능이 예술과 디자인에 미치는 영향",
        "category": "기술"
    },
    {
        "title": "AI로 그리는 세상: 이미지 생성의 미래",
        "subtitle": "

In [11]:
# 주어진 응답을 객체로 쉽게 구조화 할 수 있음
import json

python_object = json.loads(response.strip('```json\n'))
python_object[0]['title']

'이미지 생성 AI: 창의성의 새로운 지평'

#### 전략 3: 주어진 조건이 충족되었는지 확인하도록 요청

- 조건에 따른 동작을 프롬프트로 지정

- 다음 예는 주어진 텍스트를 순서대로 지시 사항으로 다시 만들 때 지시사항이 있으면 프롬프트 지시대로 만들고 만약 주어진 텍스트에 지시 사항으로 만들만한게 없으면 "제공된 단계 없음"을 출력하란 내용도 프롬프트에 포함시켜 마치 if문처럼 작동하게 만듬

- 만약 이런 조건 체크가 없으면 지시 사항으로 만들만한 내용이 없는 텍스트도 무조건 지시 사항으로 만들어 출력하므로 잘못된 출력을 얻게 됨

In [13]:
text_1 = f"""
차 한 잔을 만드는 것은 쉽습니다!
먼저 물을 끓이세요.
물이 끓는 동안 컵에 티백을 넣으세요.
물이 충분히 뜨거워지면 티백 위에 붓기만 하면 됩니다.
차가 우러날 수 있도록 잠시 기다리세요. 몇 분 후 티백을 꺼내세요.
원한다면 설탕이나 우유를 넣어 맛을 낼 수 있습니다.
쉽죠? 맛있는 차 한 잔이 완성되었습니다.
"""

prompt = f"""
세 개의 따옴표로 구분된 텍스트가 제공됩니다.
텍스트에 일련의 지침이 포함된 경우
다음 형식으로 해당 지침을 다시 작성합니다:

1단계 - ...
2단계 - ...
...
단계 N - ...

텍스트에 일련의 지침이 포함되어 있지 않은 경우
"제공된 단계 없음"이라고 입력합니다.

\"\"\"{text_1}\"\"\"
"""

print(prompt)


세 개의 따옴표로 구분된 텍스트가 제공됩니다.
텍스트에 일련의 지침이 포함된 경우
다음 형식으로 해당 지침을 다시 작성합니다:

1단계 - ...
2단계 - ...
...
단계 N - ...

텍스트에 일련의 지침이 포함되어 있지 않은 경우
"제공된 단계 없음"이라고 입력합니다.

"""
차 한 잔을 만드는 것은 쉽습니다!
먼저 물을 끓이세요.
물이 끓는 동안 컵에 티백을 넣으세요.
물이 충분히 뜨거워지면 티백 위에 붓기만 하면 됩니다.
차가 우러날 수 있도록 잠시 기다리세요. 몇 분 후 티백을 꺼내세요.
원한다면 설탕이나 우유를 넣어 맛을 낼 수 있습니다.
쉽죠? 맛있는 차 한 잔이 완성되었습니다.
"""



In [14]:
response = get_completion(prompt)
print("Completion for Text 1:")
print(response)

Completion for Text 1:
1단계 - 물을 끓입니다.
2단계 - 끓는 물을 컵에 붓고 티백을 넣습니다.
3단계 - 몇 분 동안 차를 우려냅니다.
4단계 - 티백을 제거하고 설탕이나 우유를 넣어 맛을 낼 수 있습니다.
5단계 - 맛있는 차 한 잔이 완성됩니다.


In [19]:
# 이 예제는 지시사항이 아닌데
# 프롬프트에 "제공된 단계 없음" 부분이 없으면
# 어떻게든 지시시항으로 만들어 출력하게 됨
# "제공된 단계 없음" 부분을 지우고 실험해 볼 것

text_2 = f"""
오늘은 해가 밝게 빛나고 새들이 노래하고 있습니다.
공원에서 산책하기 좋은 날입니다. 꽃이 피고 나무들이 바람에 살랑살랑 흔들리고 있습니다.
사람들은 밖으로 나와 아름다운 날씨를 즐기고 있습니다.
어떤 사람들은 피크닉을 즐기고 있고, 어떤 사람들은 게임을 하거나 잔디밭에서
휴식을 취하고 있습니다.
야외에서 시간을 보내며 자연의 아름다움을 감상하기에
자연의 아름다움을 감상하기에 완벽한 날입니다.
"""

prompt = f"""
세 개의 따옴표로 구분된 텍스트가 제공됩니다.
텍스트에 일련의 지침이 포함된 경우
다음 형식으로 해당 지침을 다시 작성합니다:

1단계 - ...
2단계 - ...
...
단계 N - ...

텍스트에 일련의 지침이 포함되어 있지 않은 경우
"제공된 단계 없음"이라고 입력합니다.

\"\"\"{text_2}\"\"\"
"""

In [20]:
response = get_completion(prompt)
print("Completion for Text 2:")
print(response)

Completion for Text 2:
"제공된 단계 없음"


#### 전략 4: 가능하면 몇개의 예제를 프롬프트에 포함 "Few-shot" prompting

- 특정 목적을 위한 챗봇을 구성하려 할때 프롬프트에 목적에 따라 지켜야 하는 조건을 만족하는 예시 몇개를 포함시켜 구체적으로 답변의 형식을 유도하는 방식

In [37]:
prompt = f"""
주어진 예의 스타일로 일관되게 대답하시오.

아이: 인내심에 대해서 가르쳐주세요.

할아버지: 가장 깊은 계곡을 깎아낸 강물은 소박한 샘에서 흐르고,
웅장한 교향곡도 음표 하나에서 시작된단다.
가장 복잡한 태피스트리는 단 한가닥의 실에서 시작되듯이
무엇이든 작은것부터 조금씩 참으며 쌓아올리는거야.

아이: 향상심에 대해서 알려주세요.
"""

response = get_completion(prompt)
print(response)



할아버지: 향상심은 자신을 끊임없이 발전시키고 성장시키는 마음가짐이야. 작은 성취부터 시작해서 조금씩 노력하고 발전해 나가는 것이 중요해. 어떤 일이든지 포기하지 않고 끝까지 노력하면 반드시 성공할 수 있어.


### Principle 2: 모델에게 "생각"할 여유를 주기

- 여러 단계로 구성된 복잡한 작업을 완성하는 하려고 한다면 완성까지 도달하는 구체적인 단계를 순서대로 지시하기


#### 전략 : 작업을 완료하는 데 필요한 단계를 지정

In [76]:
# Tom을 추가 해보기
text = f"""
In a charming village, siblings Jack, and Jill set out on
a quest to fetch water from a hilltop well.
As they climbed, singing joyfully, misfortune
struck—Jack tripped on a stone and tumbled
down the hill, with Jill following suit.
Though slightly battered, the pair returned home to
comforting embraces. Despite the mishap,
their adventurous spirits remained undimmed, and they
continued exploring with delight.
"""


In [77]:
prompt = f"""
< > 문자로 구분된 다음 텍스트를 한국어로 번역 요약하고
한국어 요약에 등장하는 이름의 수를 파악하여
다음 키가 포함된 json 객체를 출력합니다: korean_summary, num_names.

다음 형식을 사용합니다:
텍스트: < 요약할 텍스트 >
요약: 요약 원문
번역: 요약 번역
이름: 요약 번역의 이름 목록
출력 JSON: 요약 및 num_names가 포함된 json

텍스트: < {text} >
"""


In [78]:
response = get_completion(prompt)
print("\nCompletion for prompt:")
print(response)


Completion for prompt:
요약: 매력적인 마을에서 형제 자크와 질은 언덕 꼭대기 우물에서 물을 길어오기 위해 모험을 떠납니다. 자크가 돌에 걸려서 언덕을 굴러내리고 질이 따라가는데 불행이 찾아옵니다. 조금 상처를 입었지만 두 사람은 위로받으며 집으로 돌아갑니다. 사건에도 불구하고 그들의 모험 정신은 사그라들지 않고 기쁨으로 계속 탐험합니다.

이름: 자크, 질

출력 JSON: 
{
  "korean_summary": "매력적인 마을에서 형제 자크와 질은 언덕 꼭대기 우물에서 물을 길어오기 위해 모험을 떠납니다. 자크가 돌에 걸려서 언덕을 굴러내리고 질이 따라가는데 불행이 찾아옵니다. 조금 상처를 입었지만 두 사람은 위로받으며 집으로 돌아갑니다. 사건에도 불구하고 그들의 모험 정신은 사그라들지 않고 기쁨으로 계속 탐험합니다.",
  "num_names": 2
}


In [79]:
response = get_completion(prompt, model='gpt-4o')
print("\nCompletion for prompt:")
print(response)


Completion for prompt:
요약: Jack과 Jill이라는 형제가 언덕 꼭대기의 우물에서 물을 길으러 가다가 Jack이 돌에 걸려 넘어지고 Jill도 함께 굴러 내려갔다. 약간의 부상을 입었지만, 두 사람은 집으로 돌아와 위로를 받았고, 여전히 모험심을 잃지 않고 즐겁게 탐험을 계속했다.

번역: 매력적인 마을에서 형제 Jack과 Jill은 언덕 꼭대기의 우물에서 물을 길으러 나섰다. 그들이 즐겁게 노래하며 올라가던 중, Jack이 돌에 걸려 넘어지면서 언덕 아래로 굴러 떨어졌고, Jill도 뒤따라 굴러 내려갔다. 약간의 부상을 입었지만, 두 사람은 집으로 돌아와 위로를 받았고, 그들의 모험심은 여전히 꺾이지 않아 즐겁게 탐험을 계속했다.

이름: Jack, Jill

출력 JSON: {
  "korean_summary": "매력적인 마을에서 형제 Jack과 Jill은 언덕 꼭대기의 우물에서 물을 길으러 나섰다. 그들이 즐겁게 노래하며 올라가던 중, Jack이 돌에 걸려 넘어지면서 언덕 아래로 굴러 떨어졌고, Jill도 뒤따라 굴러 내려갔다. 약간의 부상을 입었지만, 두 사람은 집으로 돌아와 위로를 받았고, 그들의 모험심은 여전히 꺾이지 않아 즐겁게 탐험을 계속했다.",
  "num_names": 2
}


- 지시 사항을 순번으로 세분화

In [80]:
prompt = f"""
다음 지시 사항을 수행하세요:
1 - < > 문자로 구분된 다음 텍스트를 한 문장으로 요약합니다.
2 - 요약문을 한국어로 번역합니다.
3 - 한국어 요약에 등장하는 각 이름을 나열합니다.
4 - 다음 키가 포함된 json 객체를 출력합니다: korean_summary, num_names.

다음 형식을 사용합니다:
텍스트: < 요약할 텍스트 >
요약: 요약 원문
번역: 요약 번역
이름: 요약 번역의 이름 목록
출력 JSON: 요약 및 num_names가 포함된 json

텍스트: < {text} >
"""


In [81]:
response = get_completion(prompt)
print("\nCompletion for prompt:")
print(response)


Completion for prompt:
요약: 마을에서 사는 형제 자크와 질이 언덕 꼭대기에 있는 우물에서 물을 길어오기 위해 모험을 떠난다. 자크가 돌에 걸려서 언덕을 굴러내리고 질도 따라간다. 조금 다친 상태로 집으로 돌아와 위로받는다. 사건에도 불구하고 그들의 모험 정신은 사그라지지 않고 기쁨으로 계속 탐험한다.

번역: 마을, 자크, 질

출력 JSON:
{
  "korean_summary": "마을에서 사는 형제 자크와 질이 언덕 꼭대기에 있는 우물에서 물을 길어오기 위해 모험을 떠난다. 자크가 돌에 걸려서 언덕을 굴러내리고 질도 따라간다. 조금 다친 상태로 집으로 돌아와 위로받는다. 사건에도 불구하고 그들의 모험 정신은 사그라지지 않고 기쁨으로 계속 탐험한다.",
  "num_names": 3
}


In [82]:
response = get_completion(prompt, model='gpt-4o')
print("\nCompletion for prompt:")
print(response)


Completion for prompt:
요약: In a charming village, siblings Jack and Jill embarked on an adventure to fetch water, but after a mishap on the hill, they returned home safely and continued their explorations with enthusiasm.

번역: 매력적인 마을에서 형제 자매인 잭과 질은 물을 길으러 가는 모험을 떠났지만 언덕에서의 사고 후 안전하게 집으로 돌아와 열정적으로 탐험을 계속했습니다.

이름: 잭, 질

출력 JSON: {"korean_summary": "매력적인 마을에서 형제 자매인 잭과 질은 물을 길으러 가는 모험을 떠났지만 언덕에서의 사고 후 안전하게 집으로 돌아와 열정적으로 탐험을 계속했습니다.", "num_names": 2}


In [83]:
json.loads(response.split("출력 JSON: ")[1].strip("\n ```json"))['num_names']

2